# Gerenciando memória (rascunho)

Nesta parte, quero examinar um pouco o gerenciamento da memória do agente. Não se trata de memória ocupada, trata-se do conhecimento acumulado pelo agente, especificamente a questão de um longo histórico de diálogo, considerando a limitação de entrada dos LLMs.

## Configuração

Carregue variáveis de ambiente

In [2]:
from dotenv import load_dotenv
_ = load_dotenv()

## Acesso ao LLM

Configure o LangChain para usar o LLM desejado.

In [3]:
# from langchain_ollama import ChatOllama

# llm = ChatOllama(
#     model="llama3.2", 
#     temperature=0.0, 
#     max_tokens=1000,
# )

In [4]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="llama3-70b-8192", 
    temperature=0.0, 
    max_tokens=2000,
)

## Histórico da conversa

In [5]:
customer_info = """
- Nome: José da Silva
- Localização atual: Campinas, São Paulo
- Comidas preferidas: vegetariana
"""

Aqui a simulação de um histórico de conversa:

In [6]:
chat_history = """
Horário da conversa: 22:09:45
Data da conversa: Monday, 2025-04-28

================================ Human Message =================================

Você pode recomendar um restaurante para mim?

================================== Ai Message ==================================

Olá, José da Silva!

Infelizmente, não tenho informações específicas sobre restaurantes na sua localização atual em Campinas. Posso sugerir que você procure por opções de restaurante vegetariana no Google ou em aplicativos de entrega de comida para encontrar algo adequado às suas preferências.

Se precisar de mais ajuda, estou à disposição!

================================ Human Message =================================

Hoje é meu aniversário e quero ir a um cinema também. Aliás, eu gosto muito de cinema e teatro. Mas olha, agora estou em São José dos Campos. 

================================== Ai Message ==================================

Feliz aniversário!

São José dos Campos tem várias opções de cinemas e teatros. Você pode procurar por cinemas como o Cineplex ou o Cinemex, que oferecem uma experiência de cinema de alta qualidade.

Se precisar de mais ajuda para encontrar opções de entretenimento, estou à disposição!

================================ Human Message =================================

Bom, mas eu preciso sacar dinheiro. Eu tenho conta no banco do brasil. Tem algum Banco do Brasil aqui na cidade?

================================== Ai Message ==================================

Sim, há vários agências do Banco do Brasil em São José dos Campos. Você pode procurar por agências próximas à sua localização atual ou verificar o site do Banco do Brasil para encontrar a agência mais próxima.

Se precisar de ajuda para encontrar uma agência, posso tentar ajudar!

"""

## Resumo da conversa

Embora o tamanho máximo da entrada dos LLMs tenha aumentado significativamente, ele não é infinito e alguns LLMs ainda tem uma entrada máxima que pode ser menor que o histórico de uma longa conversa.

Uma estratégia é usar um LLM para resumir a conversa.

In [7]:
prompt = """A partir da conversa abaixo, faça um resumo dos pontos principais em apenas um parágrafo:

{history}
"""

response = llm.invoke(prompt.format(history = chat_history))

print(response.content)

Aqui está um resumo dos pontos principais da conversa: O usuário pede recomendações para um restaurante, mas o AI não tem informações específicas sobre a localização atual do usuário em Campinas, sugerindo que ele procure opções online. Em seguida, o usuário revela que está em São José dos Campos e quer ir ao cinema, então o AI sugere opções de cinemas de alta qualidade na cidade. Por fim, o usuário precisa sacar dinheiro e pergunta se há agências do Banco do Brasil em São José dos Campos, e o AI confirma que sim, sugerindo que ele procure por agências próximas ou verifique o site do banco.


## Extraindo informações

Outra abordagem é tentar gerenciar as informações mais importantes e guardá-las de forma mais estruturada.

Um LLM pode ser usado para extrair informações.

In [8]:
prompt = """Apresente a lista de informações do cliente que podem ser extraídas da conversa abaixo:

{history}
"""

response = llm.invoke(prompt.format(history = chat_history))

print(response.content)

Aqui está a lista de informações do cliente que podem ser extraídas da conversa:

1. **Nome**: José da Silva
2. **Localização atual**: São José dos Campos (anteriormente estava em Campinas)
3. **Data de nascimento**: 28 de abril de 2025 (dia da conversa)
4. **Interesses**: Cinema e teatro
5. **Necessidades**: Encontrar um restaurante, um cinema e uma agência do Banco do Brasil em São José dos Campos
6. **Banco**: Banco do Brasil (possui conta)

Essas informações podem ser úteis para personalizar a experiência do cliente e oferecer recomendações mais precisas e relevantes.


Podemos extrair as informações da conversa de maneira estruturada, usando o recurso de *structured output*.

In [10]:
from typing import List
from pydantic import BaseModel, Field
from rich.pretty import pprint


prompt = """Você deve analisar atentamente a conversa indicada abaixo.
Extraia todas as informações do cliente mencionadas nessa conversa.
Atualize as "informações anteriores"
Finalmente, retorne as informações atualizadas.

### CONVERSA

{history}

### INFORMAÇÕES ANTERIORES

{information}
"""


class CustomerInfo(BaseModel):
    nome: str = Field(description="Nome do cliente")
    data_aniversario: str = Field(
        description="Data de aniversário do cliente", default=""
    )
    endereco: str = Field(description="Endereço do cliente", default="")
    telefone: str = Field(description="Telefone do cliente", default="")
    bancos: List[str] = Field(
        description="Bancos nos quais o cliente tem conta", default=[]
    )
    localizacao_atual: str = Field(
        description="Localização atual do cliente", default=""
    )
    comidas_preferidas: List[str] = Field(description="Comidas preferidas", default=[])
    diversoes_preferidas: List[str] = Field(
        description="Diversões preferidas", default=[]
    )


llm_struct_output = llm.with_structured_output(CustomerInfo)

response = llm_struct_output.invoke(
    prompt.format(history=chat_history, information=customer_info)
)

pprint(response)

CustomerInfo(
│   nome='José da Silva',
│   data_aniversario='2025-04-28',
│   endereco='',
│   telefone='',
│   bancos=['Banco do Brasil'],
│   localizacao_atual='São José dos Campos',
│   comidas_preferidas=['vegetariana'],
│   diversoes_preferidas=['cinema', 'teatro']
)